# JavaScript 逆向爬虫实战：[Scrape Center](https://scrape.center/) - [spa6](https://spa6.scrape.center/)

## 导入依赖库

In [ ]:
import base64
import hashlib
import random
import time
from typing import List, Any

import requests
import urllib3

## 禁用安全请求警告

In [ ]:
urllib3.disable_warnings()

## 定义常量

- INDEX_URL：电影列表 URL
    - 参数
        - limit：电影数量
        - offset：偏移量
        - token：列表页 token
- DETAIL_URL：电影详情 URL
    - 参数
        - id：电影 ID（加密后）
        - token：详情页 token
- SECRET：密钥

In [ ]:
INDEX_URL = 'https://spa6.scrape.center/api/movie?limit={limit}&offset={offset}&token={token}'
DETAIL_URL = 'https://spa6.scrape.center/api/movie/{id}?token={token}'
SECRET = 'ef34#teuq0btua#(-57w1q5o5--j@98xygimlyfxs*-!i-0-mb'

## 获取 token

- 参数：path（URL 中 "/" 至 "?" 之间的部分，例如：https://spa6.scrape.center **/api/movie** /?limit=10&offset=0&token=NjY2N...E5）
- 返回：token（base64 编码后的字符串）

In [ ]:
def get_token(args: List[Any]):
    # 获取时间戳
    timestamp = str(int(time.time()))
    # 将时间戳加入参数列表
    args.append(timestamp)
    sign1 = ','.join(args)
    print('1. 拼接 path 和时间戳：\t\t', sign1)
    # 将参数列表转为字符串并进行SHA1加密
    sign2 = hashlib.sha1(sign1.encode('utf-8')).hexdigest()
    print('2. 对上一步结果进行 SHA1 加密：\t', sign2)
    # 将加密后的字符串和时间戳拼接并进行base64编码
    sign3 = sign2 + ',' + timestamp
    print('3. 拼接加密后的字符串和时间戳：\t', sign3)
    token = base64.b64encode(sign3.encode('utf-8')).decode('utf-8')
    print('4. 对上一步结果进行 base64 编码：', token, '\n')
    return token


get_token(args=['/api/movie'])

## 获取电影列表

- 参数：无
- 返回：JSON 格式的电影列表

首先，我们需要获取列表中电影的总数 count，然后根据 limit=count 构造新的 URL，最后获取全部电影列表。

In [ ]:
def get_mov_list():
    # 获取token
    token = get_token(args=['/api/movie'])
    # 构造URL
    url = INDEX_URL.format(limit=10, offset=0, token=token)
    # 获取电影总数
    mov_count = requests.get(url, verify=False).json()['count']
    # 根据电影总数构造新的URL
    url = INDEX_URL.format(limit=mov_count, offset=0, token=token)
    # 获取电影列表
    mov_list = requests.get(url, verify=False).json()
    return mov_list

get_mov_list()

## 获取电影详情

- 参数：电影 ID
- 返回：JSON 格式的电影详情

首先，将电影 ID 与密钥拼接后进行 base64 编码，得到加密后的电影 ID，然后根据加密后的电影 ID 和 token 构造新的 URL，最后获取电影详情。

In [ ]:
def get_mov_detail(mov_id):
    # 对电影ID进行加密
    encrypt_id = base64.b64encode((SECRET + str(mov_id)).encode('utf-8')).decode('utf-8')
    # 构造URL
    url = DETAIL_URL.format(id=encrypt_id, token=get_token(args=[f'/api/movie/{encrypt_id}']))
    # 获取电影详情
    return requests.get(url, verify=False).json()

## 获取所有电影详情

In [ ]:
def get_all_mov_detail(mov_list):
    # 获取所有电影详情
    all_mov_detail = [get_mov_detail(mov['id']) for mov in mov_list['results']]
    # 打印获取电影详情的数量
    print(f'已成功获取 {len(all_mov_detail)} 部电影详情')
    return all_mov_detail

In [ ]:
# 生成1-100的随机数
rid = random.randint(1, 100)
m_info = get_mov_detail(mov_id=rid)

print('电影', rid, '详情：')
for key, value in m_info.items():
    print(key + ": ", value)